# Libraries and UDFs

In [31]:
import numpy as np
import pandas as pd
import gc
import time
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scipy import stats
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer, StandardScaler

%matplotlib inline

In [210]:
def load_ponien_data(path_to_data):
    pn_train = pd.read_csv(path_to_data + '/features_train_revised.csv', index_col='SK_ID_CURR')
    pn_test = pd.read_csv(path_to_data + '/features_test_revised.csv', index_col='SK_ID_CURR')
    
    # remove infinity values
    pn_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    pn_test.replace([np.inf, -np.inf], np.nan, inplace=True)
        
    # winsorize data to avoid sensitivity to outliers
    lower = pn_train.quantile(0.005)
    upper = pn_train.quantile(0.995)
    pn_train.clip(lower, upper, axis=1, inplace=True)
    pn_test.clip(lower, upper, axis=1, inplace=True)
    
    # missing data flag
    if np.any(np.isnan(pn_train)) or np.any(np.isnan(pn_test)):
        print('Missing data in train or test set')
    else:
        print('No missing data')
    
    print('training data size:', pn_train.shape)
    print('test data size:', pn_test.shape)
    return pn_train, pn_test

In [2]:
def get_ks_scores(df_train):
    # For a training data set with binary labels, compute ks scores for each feature
    goods = df_train.query('TARGET == 0')
    defaults = df_train.query('TARGET == 1')

    start = time.time()
    variables = np.array([])
    scores = np.array([])
    for v in df_train.columns:
        variable = v
        ks = stats.ks_2samp(goods[v], defaults[v])
        variables = np.hstack((variables,v))
        scores = np.hstack((scores,ks[0]))  # large K-S statistic -> distributions different
    end = time.time() 
    print('time elapsed: {:.2f}'.format(end-start))
    return pd.Series(scores, index=variables)

def train_test_top_ks(train_data, test_data, ks_scores, top_num):
    # creates subsets of train and test data sets by selecting features according to KS statistic
    top_features = list(ks_scores.sort_values(ascending=False).index[1:top_num+1]) # 0 is TARGET
    
    df_train = train_data.copy()[top_features + ['TARGET']]
    df_test = test_data.copy()[top_features]
    print('training data size:', df_train.shape)
    print('test data size:', df_test.shape)
    return df_train, df_test

In [3]:
def drop_cols_impute_median(train_data, test_data, pct_threshold=100):
    proc_train = train_data.copy()  # to avoid overwriting original data
    proc_test = test_data.copy()
    
    # drop columns with too many missing values
    pct_missing = np.round(100*proc_train.isnull().sum()/len(proc_train), 1)
    col_missing = list(pct_missing[pct_missing>pct_threshold].index)
    
    print('deleting %d columns with %d%% or more missing data (in training set)' %(len(col_missing), pct_threshold))
    
    proc_train.drop(columns=col_missing, inplace=True)  # copy necessary otherwise original data altered!
    proc_test.drop(columns=col_missing, inplace=True)
    del pct_missing, col_missing
    gc.collect()
    
    # impute remaining missing data with median
    feature_cols = list(proc_train.columns)
    feature_cols.remove('TARGET')

    imputer = Imputer(strategy='median')
    proc_train[feature_cols]= imputer.fit_transform(proc_train[feature_cols])
    proc_test[feature_cols] = imputer.transform(proc_test[feature_cols])  # need subsetting, otherwise output is array
    print('new training data shape:', proc_train.shape)
    print('new test data shape:', proc_test.shape)

    # sanity checks    
    if np.any(np.isnan(proc_train[feature_cols])):
        raise ValueError('Missing values in new training data')
    elif np.any(np.isnan(proc_test[feature_cols])):
        raise ValueError('Missing values in new test data')
    
    return proc_train, proc_test

In [4]:
def create_train_val_set(train_data, frac_eval):
    """Create separate training and testing features and labels.
    
    Parameters
    ----------
    train_data: pandas dataframe with 'TARGET' column and feature columns (and SK_ID index)
    frac_eval: between 0 and 1. Fraction of data to be split for evaluation
    """
    X = train_data.drop(['TARGET'], axis=1)
    y = train_data['TARGET']
    
    # Create train and validation sets in stratified way
    print('Creating training and validation dataframes:')
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=frac_eval, stratify=y, random_state=42)
    print('\ttraining data size:', X_train.shape)
    print('\tvalidation data size:', X_val.shape)
    print('\tpositive labels in training data: %.4f%%' % (100*sum(y_train==1)/len(y_train)))
    print('\tpositive labels in validation data: %.4f%%' % (100*sum(y_val==1)/len(y_val)))
    return X_train, X_val, y_train, y_val

In [5]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(train_df, test_df, num_folds, stratified = False, debug= False):
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 200)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

def main(train_df, test_df,debug=False):
    with timer("Run LightGBM with kfold"):
        feat_importance = kfold_lightgbm(train_df, test_df, num_folds= 5, stratified= False, debug= debug)

# Prepare data for model fitting

In [203]:
path_to_kaggle_data='~/kaggle_JPFGM/Data/'
pn_train, pn_test = load_ponien_data(path_to_kaggle_data)

Infinity values removed from train and test data


## KS test

In [170]:
pn_train_ks_scores = get_ks_scores(pn_train)

time elapsed: 35.31


In [70]:
pn_train_ks_scores.sort_values(ascending=False).head(10)

TARGET                                                 1.000000
NEW_EXT_SOURCES_MEAN                                   0.323893
NEW_EXT_SOURCES_MEAN * num_earlypay_all_normalized     0.274700
NEW_EXT_SOURCES_MEAN * num_earlypay_1000_normalized    0.256786
EXT_SOURCE_2                                           0.223289
EXT_SOURCE_3                                           0.197463
BURO_DAYS_CREDIT_MEAN                                  0.154662
NEW_EXT_SOURCES_MEAN / num_latepay_all_normalized      0.149493
NEW_EXT_SOURCES_MEAN / num_latepay_1000_normalized     0.141983
BURO_DAYS_CREDIT_UPDATE_MEAN                           0.137266
dtype: float64

In [172]:
ks_train_600, ks_test_600 = train_test_top_ks(pn_train, pn_test, pn_train_ks_scores, 600)

training data size: (307507, 601)
test data size: (48744, 600)


In [173]:
lr_train, lr_test = drop_cols_impute_median(ks_train_600, ks_test_600, pct_threshold=60)

deleting 203 columns with 60% or more missing data (in training set)
new training data shape: (307507, 398)
new test data shape: (48744, 397)


In [179]:
# standardize data
scaler = StandardScaler()
feature_cols = list(lr_test.columns)
lr_train[feature_cols]= scaler.fit_transform(lr_train[feature_cols])  # so that output is dataframe
lr_test[feature_cols]= scaler.transform(lr_test[feature_cols])

# Logistic Regression

In [186]:
# Create train and validation set for model evaluation
X_train, X_val, y_train, y_val = create_train_val_set(lr_train, 0.2)

Creating training and validation dataframes:
	training data size: (246005, 397)
	validation data size: (61502, 397)
	positive labels in training data: 8.0730%
	positive labels in validation data: 8.0729%


In [187]:
# evaluation on val set, with faster solver
log_reg = LogisticRegression(penalty='l1', C=0.01, class_weight='balanced')
log_reg.fit(X_train, y_train)

y_predproba = log_reg.predict_proba(X_val)[:,1]
roc_auc_score(y_val, y_predproba)

0.773441605966637

## LightGBM 5 fold 600 features
Feature pre-selection according to KS statistic. Numeric features, with NA and inf

In [50]:
submission_file_name = "submission_0828_600.csv"
with timer("Full model run"):
    main(train_df=ks_train_600 , test_df=ks_test_600)

Starting LightGBM. Train shape: (307507, 600), test shape: (48744, 599)
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.776238	valid_1's auc: 0.764735


KeyboardInterrupt: 